## 코레일 기차표 크롤링 + 예매가능 기차표 찾기 자동화
https://letskorail.com/ebizprd/prdMain.do

## robots.txt 꼭 확인!
https://letskorail.com/robots.txt

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
# Selenium 웹 드라이버 서비스 설정
driver_path = ChromeDriverManager().install()
service = Service(driver_path)

url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240314&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=14&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)
wait = WebDriverWait(driver, 10) # 웹 페이지가 로드될 때까지, 최대 10초간 기다린다는 뜻

# Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
html = driver.page_source
driver.quit()

# Beautifulsoup 으로 파싱
soup = BeautifulSoup(html, 'html.parser')

In [5]:
soup

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="application/xhtml+xml; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="0725f42637afca9894a734d6ae2d7fda0bc50f2a" name="naver-site-verification"/>
<title>일반승차권(직통)</title>
<script src="/com/selectKorailCoMessage.do" type="text/javascript"></script>
<!-- 메뉴관련 -->
<script src="/js/common/korailMenu.js" type="text/javascript"></script>
<script src="/js/egovframework/korail/common/menu1.js" type="text/javascript"></script>
<script src="/js/egovframework/korail/common/menu2.js" type="text/javascript"></script>
<script src="/js/egovframework/korail/common/menu3.js" type="text/javascript"></script>
<script src="/js/egovframework/korail/common/menu4.js" type="text/javascript"></script>
<script src="/js/egovframework/korail/common/menu5.js" type="text/javascript"></script>
<scri

In [6]:
table_cotents = soup.find('tbody')
table_cotents

<tbody><tr class="">
<td>
								직통
								</td>
<td class="bdl_on bg-02" title="무궁화호">
<a href="javascript:btnTimeInfo('/ebizprd/EbizPrdTicketPr11131_i1.do?txtRunDt=20240314&amp;txtDptDt=20240314&amp;txtTrnNo=01205&amp;txtTrnGpCd=102');" title="열차운행 시각조회 새창 열기">
<span class="point-02">
									1205
									</span>
</a>
</td>
<td>
							서울<br/>07:15
							</td>
<td>
							부산<br/>12:58
							</td>
<td>
							-
							</td>
<td>
<a href="javascript:infochk(1,0);"><img alt="예약하기" name="btnRsv1_0" src="/docs/2007/img/common/icon_apm_bl.gif"/></a><p class="p5"></p><a href="javascript:infochk(6,0);" title="SeatMap예약"><img alt="좌석선택" id="btnRsv6_0" name="btnRsv6_0" src="/docs/2007/img/common/icon_apm_seatmap_yes.gif"/></a>
</td>
<td>
								-
								</td>
<td>
<img alt="역발매중" src="/docs/2007/img/common/btn_seet_rsv2.gif"/>
</td>
<!-- 365할인 -->
<td class="guide365">
<div class="shwSchPrice1"><strong>28,600원</strong></div>
</td>
<!--///////////////////////////////////////////////////

In [9]:
# 데이터를 저장할 빈 리스트 생성
data_rows = []

# table_cotents의 각 tr 태그에 대해서 반복하면서, td 데이터를 컬럼에 담아준다.
for tr in table_cotents.find_all('tr'):
    # 각 열에 해당하는 데이터 추출
    data = []
    for td in tr.find_all('td'):
        text = td.get_text(strip = True)
        data.append(text)

    # 데이터를 행으로 추가
    data_rows.append(data)

# 데이터 프레임 생성
# Column 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아', '자유석/입석',
    '인터넷 특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간'
]

df = pd.DataFrame(data_rows, columns = columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간
0,직통,1205,서울07:15,부산12:58,-,,-,,"28,600원",-,-,,,05:43
1,직통,KTX011,서울07:27,부산10:09,,,,(1량),"59,800원(5% 적립)",-,-,,,02:42
2,직통,KTX013,서울07:49,부산10:19,,,,(1량),"59,800원(5% 적립)",-,-,,,02:30
3,직통,KTX015,서울07:57,부산10:45,,,,(1량),"59,800원(5% 적립)",-,-,,,02:48
4,직통,KTX121,서울08:11,부산11:32,,,,(1량),"48,800원(5% 적립)",-,,,,03:21
5,직통,KTX191,서울08:34,부산11:03,,,,(1량),"59,800원(5% 적립)",,-,,,02:29
6,직통,1003,서울08:48,부산13:42,-,,,,"42,600원",-,-,,,04:54
7,직통,KTX017,서울08:57,부산11:15,,,,,"59,800원(5% 적립)",-,-,,,02:18
8,직통,KTX019,서울09:27,부산12:16,,,,,"59,800원(5% 적립)",,-,,,02:49
9,직통,KTX101,서울09:32,부산12:52,,,,,"53,500원(5% 적립)",-,,,,03:20


In [10]:
# 이미지 코드에 대응
# 데이터를 저장할 빈 리스트 생성
data_rows = []

# table_cotents의 각 tr 태그에 대해서 반복하면서, td 데이터를 컬럼에 담아준다.
for tr in table_cotents.find_all('tr'):
    # 각 열에 해당하는 데이터 추출
    data = []
    for td in tr.find_all('td'):
        # td 안에 img 태그가 있는지 확인, alt 속성 추출
        img_tag = td.find('img')
        # img_tag가 존재하면
        if img_tag :
            text = img_tag.get('alt', '')
        else:
            text = td.get_text(strip = True)
        
        # 열에 데이터를 추가
        data.append(text)

    # 데이터를 행으로 추가
    data_rows.append(data)

# 데이터 프레임 생성
# Column 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아', '자유석/입석',
    '인터넷 특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간'
]

df = pd.DataFrame(data_rows, columns = columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간
0,직통,1205,서울07:15,부산12:58,-,예약하기,-,역발매중,"28,600원",-,-,전동휠체어,조회,05:43
1,직통,KTX011,서울07:27,부산10:09,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:42
2,직통,KTX013,서울07:49,부산10:19,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:30
3,직통,KTX015,서울07:57,부산10:45,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48
4,직통,KTX121,서울08:11,부산11:32,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",신청하기,-,KTX-산천 AType,조회,02:29
6,직통,1003,서울08:48,부산13:42,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,전동휠체어,조회,04:54
7,직통,KTX017,서울08:57,부산11:15,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:18
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49
9,직통,KTX101,서울09:32,부산12:52,예약하기,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20


In [12]:
# URL까지 포함된 데이터 프레임 생성
# Selenium 웹 드라이버 서비스 설정
driver_path = ChromeDriverManager().install()
service = Service(driver_path)

# Selenium 웹 드라이버 실행
url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240314&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=14&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)
wait = WebDriverWait(driver, 10) # 웹 페이지가 로드될 때까지, 최대 10초간 기다린다는 뜻

# Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
html = driver.page_source
driver.quit()

# Beautifulsoup 으로 파싱
soup = BeautifulSoup(html, 'html.parser')

# table_contents로 필요한 부분만 정의
table_cotents = soup.find('tbody')

# 데이터를 저장할 빈 리스트 생성
data_rows = []

# table_cotents의 각 tr 태그에 대해서 반복하면서, td 데이터를 컬럼에 담아준다.
for tr in table_cotents.find_all('tr'):
    # 각 열에 해당하는 데이터 추출
    data = []
    for td in tr.find_all('td'):
        # td 안에 img 태그가 있는지 확인, alt 속성 추출
        img_tag = td.find('img')
        # img_tag가 존재하면
        if img_tag :
            text = img_tag.get('alt', '')
        else:
            text = td.get_text(strip = True)
        
        # 열에 데이터를 추가
        data.append(text)
    # 데이터, 즉 방금까지 td 태그들이 쌓인 data 리스트에 url도 하나 더 추가
    data.append(url)
    # 데이터를 행으로 추가
    data_rows.append(data)

# 데이터 프레임 생성
# Column 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아', '자유석/입석',
    '인터넷 특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간', 'url'
]

df = pd.DataFrame(data_rows, columns = columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,url
0,직통,1205,서울07:15,부산12:58,-,예약하기,-,역발매중,"28,600원",-,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",신청하기,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [13]:
# URL까지 포함된 데이터 프레임 생성

# XPath, '다음' 버튼 클릭, 접속된 페이지의 url을 가져오는 것까지 수행
from selenium.webdriver.common.by import By
import time

# Selenium 웹 드라이버 서비스 설정
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(executable_path=driver_path)

url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240314&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=14&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

# 5번 반복
for _ in range(5):
    # Selenium 웹 드라이버 실행
    driver.get(url) # 갱신된 url 반복적으로 전달
    time.sleep(3) # 3초 대기

    # Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
    html = driver.page_source
    

    # Beautifulsoup 으로 파싱
    soup = BeautifulSoup(html, 'html.parser')

    # table_contents로 필요한 부분만 정의
    table_cotents = soup.find('tbody')

    # 데이터를 저장할 빈 리스트 생성
    data_rows = []

    # table_cotents의 각 tr 태그에 대해서 반복하면서, td 데이터를 컬럼에 담아준다.
    for tr in table_cotents.find_all('tr'):
        # 각 열에 해당하는 데이터 추출
        data = []
        for td in tr.find_all('td'):
            # td 안에 img 태그가 있는지 확인, alt 속성 추출
            img_tag = td.find('img')
            # img_tag가 존재하면
            if img_tag :
                text = img_tag.get('alt', '')
            else:
                text = td.get_text(strip = True)
            
            # 열에 데이터를 추가
            data.append(text)
        # 데이터, 즉 방금까지 td 태그들이 쌓인 data 리스트에 url도 하나 더 추가
        data.append(url)
        # 데이터를 행으로 추가
        data_rows.append(data)

        # '다음' 버튼 찾아서 클릭
        try :
            next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[2]') # copy XPATH
        except :  # 이전 버튼이 없어서 위치가 2번째가 아니라 1번째 A 태그로 변경됨
            next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[1]') # copy XPATH
        next_button.click()
        time.sleep(3) # 3초 대기

        # 다음 버튼 누르고 이동된 페이지의 현재 url을 다시 url 변수에 넣어서 갱신
        url = driver.current_url


# 데이터 프레임 생성
# Column 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아', '자유석/입석',
    '인터넷 특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간', 'url'
]

df = pd.DataFrame(data_rows, columns = columns)
df

    
driver.quit()

KeyboardInterrupt: 

In [14]:
# 페이지네이션 : 오류 해결한 버전

# XPath, '다음' 버튼 클릭, 접속된 페이지의 url을 가져오는 것까지 수행
from selenium.webdriver.common.by import By
import time

# Selenium 웹 드라이버 서비스 설정
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(executable_path=driver_path)

url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240314&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=14&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

# 데이터를 저장할 빈 리스트 생성
data_rows = []

# 10번 반복
for _ in range(10):
    # Selenium 웹 드라이버 실행
    driver.get(url) # 갱신된 url 반복적으로 전달
    time.sleep(3) # 3초 대기

    # Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
    html = driver.page_source
    

    # Beautifulsoup 으로 파싱
    soup = BeautifulSoup(html, 'html.parser')

    # table_contents로 필요한 부분만 정의
    table_cotents = soup.find('tbody')
    
    # table_cotents의 각 tr 태그에 대해서 반복하면서, td 데이터를 컬럼에 담아준다.
    for tr in table_cotents.find_all('tr'):
        # 각 열에 해당하는 데이터 추출
        data = []
        for td in tr.find_all('td'):
            # td 안에 img 태그가 있는지 확인, alt 속성 추출
            img_tag = td.find('img')
            # img_tag가 존재하면
            if img_tag :
                text = img_tag.get('alt', '')
            else:
                text = td.get_text(strip = True)
            
            # 열에 데이터를 추가
            data.append(text)
        # 데이터, 즉 방금까지 td 태그들이 쌓인 data 리스트에 url도 하나 더 추가
        data.append(url)
        # 데이터를 행으로 추가
        data_rows.append(data)

    # '다음' 버튼 찾아서 클릭
    try :
        next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[2]') # copy XPATH
    except :  # 이전 버튼이 없어서 위치가 2번째가 아니라 1번째 A 태그로 변경됨
        next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[1]') # copy XPATH
    next_button.click()
    time.sleep(3) # 3초 대기

    # 다음 버튼 누르고 이동된 페이지의 현재 url을 다시 url 변수에 넣어서 갱신
    url = driver.current_url


# 데이터 프레임 생성
# Column 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아', '자유석/입석',
    '인터넷 특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간', 'url'
]

df = pd.DataFrame(data_rows, columns = columns)
df

    
driver.quit()

In [17]:
df.head(14)

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,url
0,직통,1205,서울07:15,부산12:58,-,예약하기,-,역발매중,"28,600원",-,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",신청하기,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [18]:
df = df.drop_duplicates(subset = ['출발시각'], keep = 'first')
df.head(20)

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,url
0,직통,1205,서울07:15,부산12:58,-,예약하기,-,역발매중,"28,600원",-,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",신청하기,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [21]:
fast_reserve_df = df[df['일반실'] == '예약하기']
# 인덱스 리셋
fast_reserve_df = fast_reserve_df.reset_index(drop = True)
fast_reserve_df.head(20)

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷 특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,url
0,직통,1205,서울07:15,부산12:58,-,예약하기,-,역발매중,"28,600원",-,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,1003,서울08:48,부산13:42,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,KTX017,서울08:57,부산11:15,좌석매진,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX101,서울09:32,부산12:52,좌석매진,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX021,서울09:57,부산12:45,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX123,서울10:11,부산13:34,예약하기,예약하기,유아동반객실,역발매중,"48,800원(5% 적립)",-,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:23,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [22]:
fast_reserve_df.loc[0, 'url']

'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240314&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=14&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

In [23]:
fast_reserve_df.to_csv('korail_fast_reserve.csv', index = False)